In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
elif PROJECT_ROOT.name.startswith("0"):
    PROJECT_ROOT = PROJECT_ROOT.parent.parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

%load_ext autoreload
%autoreload 2

In [2]:
# !pip install shap
# !pip install hyperopt
# !pip install loguru
# !pip install lightgbm
# !pip install catboost

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Data Management
import json
import numpy as np
import pandas as pd

# Modelling
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
import xgboost as xgb
import lightgbm as lgb

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from match_forecast.utils import *

2025-04-17 22:59:45.943 | INFO     | match_forecast.config:<module>:11 - PROJ_ROOT path is: /Users/maichoun/QRT-Challenge-2024


In [4]:
PROJ_ROOT = Path("QRT-Challenge-2024").resolve().parents[1]
DATA_DIR = PROJ_ROOT / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
PROCESSED_DATA_DIR = DATA_DIR / "processed"

try:
    train_data = pd.read_csv(PROCESSED_DATA_DIR / "train_data.csv", index_col=0)
    train_scores = pd.read_csv(RAW_DATA_DIR / "Y_train.csv", index_col=0)
    print("Files loaded")
    
except FileNotFoundError as e:
    print(e)

Files loaded


In [5]:
train_data.shape

(12303, 956)

In [6]:
train_scores = train_scores.loc[train_data.index]
train_scores_1c = train_scores[['HOME_WINS', 'DRAW', 'AWAY_WINS']].idxmax(axis=1)
label_mapping = {'HOME_WINS': 0, 'DRAW': 1, 'AWAY_WINS': 2}
train_scores_1c = train_scores_1c.replace(label_mapping)

train_scores_1c.head(5)

ID
0    2
1    1
2    2
3    0
4    1
dtype: int64

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_scores_1c, train_size=0.8, random_state=42)

In [8]:
with open("xgb_model.json", "r") as f:
    results_with_params = json.load(f)

print("\nBest trial")
best_trial = min(results_with_params, key=lambda x: x["loss"])
print("xgb:", best_trial["params"])
print("Best loss:", best_trial["loss"])


Best trial
xgb: {'colsample_bytree': 0.6831357548929327, 'gamma': 3.4044955134548194, 'learning_rate': 0.008412006493982379, 'max_depth': 7.0, 'min_child_weight': 6.0, 'n_estimators': 500.0, 'reg_alpha': 0.6290909433899785, 'reg_lambda': 0.41575585612788596, 'subsample': 0.8744674667387885}
Best loss: 0.5013201780143373


In [9]:
best_trial["params"]['max_depth'] = int(best_trial["params"]['max_depth'])
best_trial["params"]['n_estimators'] = int(best_trial["params"]['n_estimators'])
best_trial["params"]['min_child_weight'] = int(best_trial["params"]['min_child_weight'])

xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    n_jobs=-1,
    **best_trial["params"]
)

In [10]:
with open("xt_model.json", "r") as f:
    results_with_params = json.load(f)

print("\nBest trial")
best_trial = min(results_with_params, key=lambda x: x["loss"])
print("rf:", best_trial["params"])
print("Best loss:", best_trial["loss"])


Best trial
rf: {'class_weight': 0, 'criterion': 1, 'max_depth': 10.0, 'max_features': 0.6268856852083861, 'max_leaf_nodes': 600.0, 'min_impurity_decrease': 0.0015146957197272166, 'min_samples_leaf': 2.0, 'min_samples_split': 16.0, 'n_estimators': 950.0}
Best loss: 0.5067054896013758


In [11]:
best_trial['params']['n_estimators'] = int(best_trial['params']['n_estimators'])
best_trial['params']['max_depth'] = int(best_trial['params']['max_depth'])
best_trial['params']['min_samples_split'] = int(best_trial['params']['min_samples_split'])
best_trial['params']['min_samples_leaf'] = int(best_trial['params']['min_samples_leaf'])
best_trial['params']['max_leaf_nodes'] = int(best_trial['params']['max_leaf_nodes']) if best_trial['params']['max_leaf_nodes'] is not None else None

class_weight_options = [None, 'balanced', 'balanced_subsample']
if isinstance(best_trial['params']['class_weight'], (int, np.integer)):  # Ensure it's an index
    best_trial['params']['class_weight'] = class_weight_options[int(best_trial['params']['class_weight'])]


criterion_options = ['gini', 'entropy']
if isinstance(best_trial['params']['criterion'], (int, np.integer)):
    best_trial['params']['criterion'] = criterion_options[int(best_trial['params']['criterion'])]

xt_model = ExtraTreesClassifier(
    **best_trial['params'],
    random_state=42,
    n_jobs=-1
)

In [12]:
with open("lgb_model.json", "r") as f:
    results_with_params = json.load(f)

print("\nBest trial")
best_trial = min(results_with_params, key=lambda x: x["loss"])
print("lgb:", best_trial["params"])
print("Best loss:", best_trial["loss"])


Best trial
lgb: {'bagging_fraction': 0.8837274787818103, 'feature_fraction': 0.6253342690722408, 'lambda_l1': 0.5072607592076765, 'lambda_l2': 0.3193986309883332, 'learning_rate': 0.002252648901581115, 'max_depth': 6.0, 'min_child_samples': 80.0, 'n_estimators': 700.0, 'num_leaves': 40.0}
Best loss: 0.5062995430701032


In [13]:
best_trial["params"]['max_depth'] = int(best_trial["params"]['max_depth'])
best_trial["params"]['num_leaves'] = int(best_trial["params"]['num_leaves'])
best_trial["params"]['n_estimators'] = int(best_trial["params"]['n_estimators'])
best_trial["params"]['min_child_samples'] = int(best_trial["params"]['min_child_samples'])

lgb_model = lgb.LGBMClassifier(
    objective='multiclass', 
    boosting_type='gbdt', 
    num_class=3,
    n_jobs=-1, 
    **best_trial["params"] 
)

In [14]:
with open("rf_model.json", "r") as f:
    results_with_params = json.load(f)

print("\nBest trial")
best_trial = min(results_with_params, key=lambda x: x["loss"])
print("Best hyperparameters:", best_trial["params"])
print("Best loss:", best_trial["loss"])


Best trial
Best hyperparameters: {'bootstrap': 0, 'class_weight': 0, 'criterion': 0, 'max_depth': 28.0, 'max_features': 0.4659889990637031, 'max_leaf_nodes': 230.0, 'min_impurity_decrease': 0.001020132645788404, 'min_samples_leaf': 14.0, 'min_samples_split': 42.0, 'n_estimators': 1800.0}
Best loss: 0.5032507316082


In [15]:
best_trial['params']['n_estimators'] = int(best_trial['params']['n_estimators'])
best_trial['params']['max_depth'] = int(best_trial['params']['max_depth'])
best_trial['params']['min_samples_split'] = int(best_trial['params']['min_samples_split'])
best_trial['params']['min_samples_leaf'] = int(best_trial['params']['min_samples_leaf'])
best_trial['params']['max_leaf_nodes'] = int(best_trial['params']['max_leaf_nodes']) if best_trial['params']['max_leaf_nodes'] is not None else None
best_trial['params']['bootstrap'] = bool(best_trial['params']['bootstrap'])

class_weight_options = [None, 'balanced', 'balanced_subsample']
if isinstance(best_trial['params']['class_weight'], (int, np.integer)):  # Ensure it's an index
    best_trial['params']['class_weight'] = class_weight_options[int(best_trial['params']['class_weight'])]

criterion_options = ['gini', 'entropy']
if isinstance(best_trial['params']['criterion'], (int, np.integer)):
    best_trial['params']['criterion'] = criterion_options[int(best_trial['params']['criterion'])]


rf_model = RandomForestClassifier(
    **best_trial['params'],
    n_jobs=-1,
    random_state=42
)

In [16]:
with open("catboost_model.json", "r") as f:
    results_with_params = json.load(f)

print("\nBest trial")
best_trial = min(results_with_params, key=lambda x: x["loss"])
print("Best hyperparameters:", best_trial["params"])
print("Best loss:", best_trial["loss"])


Best trial
Best hyperparameters: {'auto_class_weights': 0, 'bagging_temperature': 0.19814244495423616, 'border_count': 128.0, 'depth': 9.0, 'grow_policy': 0, 'iterations': 550.0, 'l2_leaf_reg': 4.707002220960229, 'learning_rate': 0.01835658452772701, 'random_strength': 3.9813457467596693, 'rsm': 0.5158050292260733}
Best loss: 0.5021328454293381


In [17]:
best_params = best_trial['params']

# Cast integer‐only hyperparameters
for int_param in ('iterations', 'depth', 'border_count'):
    if int_param in best_params:
        best_params[int_param] = int(best_params[int_param])

# Map grow_policy index → string
grow_policy_options = ['SymmetricTree', 'Depthwise', 'Lossguide']
gp = best_params.get('grow_policy')
if isinstance(gp, (int, np.integer)):
    best_params['grow_policy'] = grow_policy_options[int(gp)]

# Map auto_class_weights index → None or 'Balanced'
auto_weights_options = [None, 'Balanced']
aw = best_params.get('auto_class_weights')
if isinstance(aw, (int, np.integer)):
    best_params['auto_class_weights'] = auto_weights_options[int(aw)]

catboost_model = CatBoostClassifier(
    loss_function='MultiClass',
    eval_metric='MultiClass',
    verbose=False,
    thread_count=-1,
    random_seed=42,
    **best_params
)    

In [18]:
with open("logreg_model.json", "r") as f:
    results_with_params = json.load(f)

print("\nBest trial")
best_trial = min(results_with_params, key=lambda x: x["loss"])
print("lr:", best_trial["params"])
print("Best loss:", best_trial["loss"])


Best trial
lr: {'C': 3.6759085670934373, 'l1_ratio': 0.701407190130335, 'n_components': 20.0}
Best loss: 0.4997963350634015


In [19]:
best_trial['params']['n_components'] = int(best_trial['params']['n_components'])

logreg_model = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    C=best_trial["params"]['C'],
    l1_ratio=best_trial["params"]['l1_ratio'],
    max_iter=2000,
    random_state=42
)

In [20]:
logreg_pipeline = Pipeline([
    ('scaler1', StandardScaler()),
    ('pca', PCA(n_components=best_trial["params"]['n_components'])),  
    ('scaler2', StandardScaler()),  
    ('lr', logreg_model)  
])

In [21]:
lgb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
xt_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
catboost_model.fit(X_train, y_train)
logreg_pipeline.fit(X_train, y_train)

[LightGBM] [Warning] lambda_l1 is set=0.5072607592076765, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5072607592076765
[LightGBM] [Warning] feature_fraction is set=0.6253342690722408, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6253342690722408
[LightGBM] [Warning] bagging_fraction is set=0.8837274787818103, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8837274787818103
[LightGBM] [Warning] lambda_l2 is set=0.3193986309883332, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3193986309883332
[LightGBM] [Warning] lambda_l1 is set=0.5072607592076765, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5072607592076765
[LightGBM] [Warning] feature_fraction is set=0.6253342690722408, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6253342690722408
[LightGBM] [Warning] bagging_fraction is set=0.8837274787818103, subsample=1.0 will be ignored. Current value: bagging_fraction=0.883727478781810

Pipeline(steps=[('scaler1', StandardScaler()), ('pca', PCA(n_components=20)),
                ('scaler2', StandardScaler()),
                ('lr',
                 LogisticRegression(C=3.6759085670934373,
                                    l1_ratio=0.701407190130335, max_iter=2000,
                                    penalty='elasticnet', random_state=42,
                                    solver='saga'))])

In [25]:
from itertools import combinations

model_preds = {
    "logreg": logreg_pipeline.predict_proba(X_test),
    "xgb": xgb_model.predict_proba(X_test),
    "lgb": lgb_model.predict_proba(X_test),
    "xt": xt_model.predict_proba(X_test),
    "rf": rf_model.predict_proba(X_test),
    "catboost": catboost_model.predict_proba(X_test)

}

models = list(model_preds.keys())

combi_results = {}

for combi in combinations(models, 5):
    combi_name = "_".join(combi)
    
    avg_pred = np.mean([model_preds[m] for m in combi], axis=0)  
    
    y_pred_final = np.argmax(avg_pred, axis=1)

    final_accuracy = accuracy_score(y_test, y_pred_final)
    
    combi_results[combi_name] = final_accuracy
    
    print(f"Combination: {combi}, Accuracy (test) : {final_accuracy * 100:.2f}%")

best_combi = max(combi_results, key=combi_results.get)
print(f"\nBest Combination: {best_combi}, Accuracy: {combi_results[best_combi] * 100:.2f}%")


[LightGBM] [Warning] lambda_l1 is set=0.5072607592076765, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5072607592076765
[LightGBM] [Warning] feature_fraction is set=0.6253342690722408, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6253342690722408
[LightGBM] [Warning] bagging_fraction is set=0.8837274787818103, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8837274787818103
[LightGBM] [Warning] lambda_l2 is set=0.3193986309883332, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3193986309883332
Combination: ('logreg', 'xgb', 'lgb', 'xt', 'rf'), Accuracy (test) : 49.86%
Combination: ('logreg', 'xgb', 'lgb', 'xt', 'catboost'), Accuracy (test) : 50.26%
Combination: ('logreg', 'xgb', 'lgb', 'rf', 'catboost'), Accuracy (test) : 50.30%
Combination: ('logreg', 'xgb', 'xt', 'rf', 'catboost'), Accuracy (test) : 50.43%
Combination: ('logreg', 'lgb', 'xt', 'rf', 'catboost'), Accuracy (test) : 50.35%
Combination: ('xgb', 'lgb', 'x